In [3]:
import torch
torch.LongTensor([[1,2,4,5],[4,3,2,9]]).shape

torch.Size([2, 4])

In [ ]:
import torch
import sys
from torch import nn, optim
import torch.nn.functional as F
from packages.functions import num_to_var
import numpy as np
from collections import Counter
from torch.autograd import Variable
import time

class Attention(nn.Module):
    r"""
    Applies an attention mechanism on the output features from the decoder.
    .. math::
            \begin{array}{ll}
            x = context*output \\
            attn = exp(x_i) / sum_j exp(x_j) \\
            output = \tanh(w * (attn * context) + b * output)
            \end{array}
    Args:
        dim(int): The number of expected features in the output
    Inputs: output, context
        - **output** (batch, output_len, dimensions): tensor containing the output features from the decoder.
        - **context** (batch, input_len, dimensions): tensor containing features of the encoded input sequence.
    Outputs: output, attn
        - **output** (batch, output_len, dimensions): tensor containing the attended output features from the decoder.
        - **attn** (batch, output_len, input_len): tensor containing attention weights.
    Attributes:
        linear_out (torch.nn.Linear): applies a linear transformation to the incoming data: :math:`y = Ax + b`.
        mask (torch.Tensor, optional): applies a :math:`-inf` to the indices specified in the `Tensor`.
    Examples::
         >>> attention = seq2seq.models.Attention(256)
         >>> context = Variable(torch.randn(5, 3, 256))
         >>> output = Variable(torch.randn(5, 5, 256))
         >>> output, attn = attention(output, context)
    """
    def __init__(self, dim):
        super(Attention, self).__init__()
        self.linear_out = nn.Linear(dim*2, dim)
        self.mask = None

    def set_mask(self, mask):
        """
        Sets indices to be masked
        Args:
            mask (torch.Tensor): tensor containing indices to be masked
        """
        self.mask = mask

    def forward(self, output, context):
        batch_size = output.size(0)
        hidden_size = output.size(2)
        input_size = context.size(1)
        # (batch, out_len, dim) * (batch, in_len, dim) -> (batch, out_len, in_len)
        attn = torch.bmm(output, context.transpose(1, 2))
        if self.mask is not None:
            attn.data.masked_fill_(self.mask, -float('inf'))
        attn = F.softmax(attn.view(-1, input_size)).view(batch_size, -1, input_size)

        # (batch, out_len, in_len) * (batch, in_len, dim) -> (batch, out_len, dim)
        mix = torch.bmm(attn, context)

        # concat -> (batch, out_len, 2*dim)
        combined = torch.cat((mix, output), dim=2)
        # output -> (batch, out_len, dim)
        output = F.tanh(self.linear_out(combined.view(-1, 2 * hidden_size))).view(batch_size, -1, hidden_size)

        return output, attn
    
class Model(nn.Module):
    def __init__(self, args,config):
        super(Model, self).__init__()
        
        #Arguments
        
        
        #Model Requirements
        self.embed = nn.Embedding(vocab_size, embed_size)
        if initial_embedding is not None:
                    #TODO initialize embeddings with the embeddings received also decide trainable or not trainable
        else:
            #TODO randomly initialize embeddings
        
        self.projection = nn.Linear(hidden_size, len_vocab)
        self.cell_encoder = nn.GRU(input_size=embed_size,
            hidden_size=hidden_size, batch_first=True,
            bidirectional=True)
        if config.config_dir["same_cell"] == True:
            self.cell_query = self.cell_encoder
        else:
            self.cell_query = nn.GRU(input_size=embed_size,
                hidden_size=hidden_size, batch_first=True,
                bidirectional=True)
            
        self.cell_decoder = nn.LSTM(input_size=embed_size,
            hidden_size=hidden_size, batch_first=True)
        self.DistractionDecoder = Decoder(TODO, parameters)
        if (config.config_dir["is_bidir"]):
        	hs = 2*hidden_size
        else:
        	hs = hidden_size
        self.attention = Attention(hidden_size*2, hidden_size)
        if config.config_dir["distraction_cell"] == "LSTM_soft":
        	distract_cell = DistractionLSTMCell_soft(hs, state_is_tuple = True)
        elif config.config_dir["distraction_cell"] == "LSTM_hard":
        	distract_cell = DistractionLSTMCell_hard(hs, state_is_tuple=True)
        elif config.config_dir["distraction_cell"] == "LSTM_sub":
        	distract_cell = DistractionLSTMCell_subtract(hs, state_is_tuple=True)
        elif config.config_dir["distraction_cell"] == "GRU_hard":
        	distract_cell = DistractionGRUCell_hard(hs) 
        elif config.config_dir["distraction_cell"] == "GRU_soft":
        	distract_cell = DistractionGRUCell_soft(hs)
        elif config.config_dir["distraction_decoder_start_cell"] == "GRU_sub":
        	distract_cell = DistractionGRUCell_subtract(hs)

        

    def forward(self, encoder_inputs,query_inputs, target, batch, vocab, train=True):
        encoder_embedded_inputs = self.embed(encode_inputs)
        query_embedded_inputs = self.embed(query_inputs)
        encoder_outputs, encoder_state = self.cell_encoder(encoder_embedded_inputs)
        query_outputs, query_state = self.cell_query(query_embedded_inputs)
        
        d = encoder_outputs[-1]
        ey =  
        for i, inp in enumerate(target):
            _, s = self.cell_decoder(s, )
        
        # TODO Do Attention Calculation as required code examples from BIMPM or Gettothepoint
        
        
        # TODO Decoder and inside Decoder - Diversity
        
        return out_list, cov_loss

    def mask(self, matrix):
        # for obtaining a FloatTensor Variable mask out of a LongTensor Variable
        out = matrix==0
        return Variable(out.float().data)

    def to_cuda(self, tensor):
        # turns to cuda
        if torch.cuda.is_available():
            return tensor.cuda()
        else:
            return tensor

